In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from datetime import datetime
import shap
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import mifs
from boruta import BorutaPy

# Assuming df is your DataFrame after aggregating to daily data and merging with weather/building data
df = pd.read_csv("Data.csv")# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# Get the number of rows and columns 
# rows = len(df.axes[0]) 
# cols = len(df.axes[1]) 
df.info()

c:\AOB\AOB_Energy_Consumption_Analysis_Prediction\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41666 entries, 0 to 41665
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   time                               41664 non-null  object 
 1   Comms and Services                 40812 non-null  float64
 2   Car Chargers                       40812 non-null  float64
 3   Space Heating                      40812 non-null  float64
 4   Hot Water                          40812 non-null  float64
 5   Sockets                            40812 non-null  float64
 6   Lighting                           40812 non-null  float64
 7   total_energy                       41664 non-null  object 
 8   datepart                           41664 non-null  float64
 9   weekend                            41664 non-null  object 
 10  bank holiday                       41664 non-null  object 
 11  hour                               41664 non-null  flo

C:\Users\mscVRstudent22\AppData\Local\Temp\ipykernel_13704\2966589377.py:20: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data.csv")# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"


In [2]:
# Generate a list of columns to drop
columns_to_drop = [col for col in df.columns if col.startswith('observation')]

# Drop these columns from the DataFrame
df = df.drop(columns=columns_to_drop)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40131 entries, 112 to 41663
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   time                               40131 non-null  object 
 1   Comms and Services                 40131 non-null  float64
 2   Car Chargers                       40131 non-null  float64
 3   Space Heating                      40131 non-null  float64
 4   Hot Water                          40131 non-null  float64
 5   Sockets                            40131 non-null  float64
 6   Lighting                           40131 non-null  float64
 7   total_energy                       40131 non-null  object 
 8   datepart                           40131 non-null  float64
 9   weekend                            40131 non-null  object 
 10  bank holiday                       40131 non-null  object 
 11  hour                               40131 non-null  float6

In [ ]:
from sklearn.preprocessing import SplineTransformer

def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )

# Apply Spline Transformation
def apply_spline_transformer(df, column, period):
    spline_transformer = periodic_spline_transformer(period)
    transformed_values = spline_transformer.fit_transform(df[[column]])
    
    # Create new column names for the transformed features
    transformed_cols = [f"{column}_spline_{i}" for i in range(transformed_values.shape[1])]
    df[transformed_cols] = transformed_values
    
    return df

In [ ]:
# Apply spline transformation for the temporal features
df = apply_spline_transformer(df, 'month', 12)
df = apply_spline_transformer(df, 'hour', 24)
df = apply_spline_transformer(df, 'day of week', 7)
df = apply_spline_transformer(df, 'day of month', 31)

# Additional feature engineering
df['working_hours'] = df['hour'].apply(lambda x: 8 <= x <= 17)
df['bank holiday'] = df['bank holiday'].astype(int)
df['weekend'] = df['weekend'].astype(int)

# Drop the original columns if no longer needed
df.drop(columns=['month', 'hour', 'day of week', 'day of month'], inplace=True)

# Display the updated DataFrame
print(df.head())

In [ ]:
columns_to_convert = ['weekend','bank holiday']

# Convert 0 and 1 to boolean values in specified columns
df[columns_to_convert] = df[columns_to_convert].astype(bool)
# Sum specified columns to create 'total_aob_energy'
df['total_aob_energy'] = df[[ 'Hot Water', 'Sockets', 'Lighting','Comms and Services', 'Space Heating']].sum(axis=1, skipna=True) # type: ignore

# Drop the original columns except 'Car Chargers'
df.drop(['Comms and Services', 'Space Heating', 'Hot Water', 'Sockets', 'Lighting' ], axis=1, inplace=True)

# Now, 'df' contains the new 'total_aob_energy' column and has the specified columns dropped, except 'Car Chargers'

In [ ]:
df['time'] = pd.to_datetime(df['time'])
df = df.dropna()
df = df.select_dtypes(exclude=['object'])
df.columns = [col.capitalize() for col in df.columns]
print(df.columns)
df.info()

In [ ]:
df = df.drop(['Car chargers','Datepart'], axis = 1)

In [ ]:
new_columns = []
for col in df.columns:
    # Remove "Forecast_" if it exists and capitalize the first letter of the remaining string
    if col.startswith('Forecast_'):
        new_name = col.replace('Forecast_', '')  # Remove 'Forecast_'
        new_name = new_name.capitalize()  # Capitalize the first letter
    else:
        new_name = col  # Keep the original name if it doesn't start with 'Forecast_'
    new_columns.append(new_name)

# Assign the modified column names back to the DataFrame
df.columns = new_columns


In [ ]:
# Set 'date' column as the DataFrame index
df.set_index('Time', inplace=True)
#removing precovid data
df = df[df.index >= "2021-01-01"]

In [ ]:
# Remove rare/uncommon values (assuming they are below a threshold or beyond a threshold)
df = df[(df['Total_aob_energy'] > df['Total_aob_energy'].quantile(0.01)) & 
        (df['Total_aob_energy'] < df['Total_aob_energy'].quantile(0.99))]

train_data_split1 = df[((df.index.year == 2021) | (df.index.year == 2022)) | 
                       ((df.index.year == 2023) & (df.index.day % 28 <= 7) | (df.index.day % 28 > 14) & (df.index.day % 28 <= 21))]
test_data_split1 = df[(df.index.year == 2023) & (df.index.day % 28 > 7) & (df.index.day % 28 <= 14) | (df.index.day % 28 > 21)]

# Split 2: First and third weeks of each month for testing, second and fourth weeks for training
train_data_split2 = df[((df.index.year == 2021) | (df.index.year == 2022)) | 
                       ((df.index.year == 2023) & (df.index.day % 28 > 7) & (df.index.day % 28 <= 14) | (df.index.day % 28 > 21))]
test_data_split2 = df[(df.index.year == 2023) & (df.index.day % 28 <= 7) | (df.index.day % 28 > 14) & (df.index.day % 28 <= 21)]

# Display the shapes of the splits to verify
print("Split 1 - Train set shape:", train_data_split1.shape)
print("Split 1 - Test set shape:", test_data_split1.shape)
print("Split 2 - Train set shape:", train_data_split2.shape)
print("Split 2 - Test set shape:", test_data_split2.shape)

# Split data into features and target for Split 1
X_train_split1 = train_data_split1.drop(['Total_aob_energy'], axis=1)
y_train_split1 = train_data_split1['Total_aob_energy']
X_test_split1 = test_data_split1.drop(['Total_aob_energy'], axis=1)
y_test_split1 = test_data_split1['Total_aob_energy']

# Split data into features and target for Split 2
X_train_split2 = train_data_split2.drop(['Total_aob_energy'], axis=1)
y_train_split2 = train_data_split2['Total_aob_energy']
X_test_split2 = test_data_split2.drop(['Total_aob_energy'], axis=1)
y_test_split2 = test_data_split2['Total_aob_energy']
